**Initialization**


---



Imports

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

Hyperparameters

In [ ]:
n_epoch = 250
batch_size = 50
threshold = 128
learning_rate = 0.0001


input_feature_dimension = 2
hidden_feature_dimension = 5
output_feature_dimension = 10

Import MNIST

In [ ]:
mnist = keras.datasets.mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

**Data formatting**


---



Methods for formatting the dataset

In [ ]:
def format_data_conventional(image):
  data = np.zeros((28*28), dtype=int)
  for i in range(28):
    for j in range(28):
      if image[i][j] > threshold:
        data[j*28+i] = 1
  return data

def generate_images_conventional(images):
  image_array = ([])
  for i in range(len(images)):
    image_array.append(format_data_conventional(images[i]))
  return np.array(image_array)

def format_label(labels):
    label_array = []
    for i in range(len(labels)):
        label = np.zeros(10)
        label[labels[i]] = 1
        label_array.append(label)
    return np.array(label_array)

Calculate training data and test data using the previously defined methods

In [ ]:
training_data_conventional = generate_images_conventional(training_images)
test_data_conventional = generate_images_conventional(test_images)
training_label = format_label(training_labels)
test_label = format_label(test_labels)

In [ ]:
print("Training data shape  = ", training_data_conventional.shape,
     "\nTest data shape      = ", test_data_conventional.shape,
     "\nTraining label shape = ", training_label.shape,
     "\nTest label shape     = ", test_label.shape)

**The Network**


---



Structure

In [ ]:
# Update this string when altering the network
network_stats = '128-128-10'

# Initialization of input, output and mask shapes
x = tf.placeholder(tf.float32, [None, (28*28)])
y = tf.placeholder(tf.float32, [None, output_feature_dimension])

# The network:

input_layer = tf.layers.dense(inputs=x, units=128, activation=tf.nn.relu)
hidden_layer_1 = \
  tf.layers.dense(inputs=input_layer, units = 128, activation=tf.nn.relu)
output_layer = \
tf.layers.dense(inputs=hidden_layer_1, units=output_feature_dimension)

output_softmax = tf.nn.softmax(output_layer)
y_pred = tf.argmax(output_layer, axis=1)

Loss function and optimizer is defined

In [ ]:
loss = \
  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, 
                                                            logits=output_layer))
optimize = tf.train.AdamOptimizer(learning_rate).minimize(loss)

A method for calculating the correct prediction percentage of the network is defined here.

In [ ]:
prediction_array = ([])
def predictions(output, labels):
    correct = 0;
    for i in range (output.size):
        if output[i] == labels[i]:
            correct = correct + 1
    percentage = str(correct/output.size*100)
    prediction_array.append(percentage)
    return ("Correct: " + percentage + "%")

**Running the network**


---



Training the network

In [ ]:
with tf.train.MonitoredSession() as sess:
  for epoch in range(n_epoch):

    for i in range(training_data_conventional.shape[0]//batch_size):
      sess.run(optimize, 
               {x: training_data_conventional[i*batch_size:(i+1)*batch_size], 
                y: training_label[i*batch_size:(i+1)*batch_size]})

      test_loss = \
        sess.run(loss, 
                 {x:training_data_conventional[i*batch_size:(i+1)*batch_size], 
                  y: training_label[i*batch_size:(i+1)*batch_size]})
            
      y_val, y_pred_val = \
        sess.run([output_softmax, y_pred], {x: test_data_conventional})
      
    print('Epoch: ' , epoch, ' - Loss: ', str(test_loss))
        
    if ((epoch % 1) == 0):
     pred = predictions(y_pred_val, test_labels)
     print(pred)

print("Done")

Displaying the results

In [ ]:
ID_string = \
    str(network_stats) + \
    ".-E" + str(n_epoch) + \
    "-BS" + str(batch_size) + \
    "-LR" + str(learning_rate) + \
    "-TH" + str(threshold)
print(ID_string)
print('Test loss:', test_loss)
predictions(y_pred_val, test_labels)